In [1]:
#imports

!pip install ipython-cache
import cache_magic
from random import randint
import sys
import math
import json
%matplotlib inline
import matplotlib.pyplot as plt
import math
from math import exp, factorial
import numpy as np



%cache magic is now registered in ipython


In [ ]:
def printMatrix(matrix):
    for r in matrix:
        print(r)
    print()

In [ ]:

def GetNumberPoisson(l, n):
    return math.pow(l, n) / math.factorial(n) * math.exp(-l)

probs3 = [GetNumberPoisson(3, n) for n in range(100)]
probs4 = [GetNumberPoisson(4, n) for n in range(100)]
ns = [n for n in range(100)]

s1 = 0
for n in range(20):
    s1 = s1 + GetNumberPoisson(3, n)

s2 = 0
for n in range(20):
    s2 = s2 + GetNumberPoisson(4, n)

print (s1, s2)

plt.plot(ns, probs3, 'r')
plt.plot(ns, probs4, 'b')

In [ ]:
# maximum # of cars in each location
MAX_CARS = 20

# maximum # of cars to move during night
MAX_MOVE_OF_CARS = 5

# expectation for rental requests in first location
RENTAL_REQUEST_FIRST_LOC = 3

# expectation for rental requests in second location
RENTAL_REQUEST_SECOND_LOC = 4

# expectation for # of cars returned in first location
RETURNS_FIRST_LOC = 3

# expectation for # of cars returned in second location
RETURNS_SECOND_LOC = 2

DISCOUNT = 0.9

# credit earned by a car
RENTAL_CREDIT = 10

# cost of moving a car
MOVE_CAR_COST = 2

# all possible actions
actions = np.arange(-MAX_MOVE_OF_CARS, MAX_MOVE_OF_CARS + 1)

# An up bound for poisson distribution
# If n is greater than this value, then the probability of getting n is truncated to 0
POISSON_UPPER_BOUND = 11


def expected_return(state, action, state_value, constant_returned_cars):
    # initailize total return
    returns = 0.0

    # cost for moving cars
    returns -= MOVE_CAR_COST * abs(action)

    # go through all possible rental requests
    for rental_request_first_loc in range(0, POISSON_UPPER_BOUND):
        for rental_request_second_loc in range(0, POISSON_UPPER_BOUND):
            # moving cars
            num_of_cars_first_loc = int(min(state[0] - action, MAX_CARS))
            num_of_cars_second_loc = int(min(state[1] + action, MAX_CARS))

            # valid rental requests should be less than actual # of cars
            real_rental_first_loc = min(num_of_cars_first_loc, rental_request_first_loc)
            real_rental_second_loc = min(num_of_cars_second_loc, rental_request_second_loc)

            # get credits for renting
            reward = (real_rental_first_loc + real_rental_second_loc) * RENTAL_CREDIT
            num_of_cars_first_loc -= real_rental_first_loc
            num_of_cars_second_loc -= real_rental_second_loc

            # probability for current combination of rental requests
            prob = poisson(rental_request_first_loc, RENTAL_REQUEST_FIRST_LOC) * \
                         poisson(rental_request_second_loc, RENTAL_REQUEST_SECOND_LOC)

            if constant_returned_cars:
                # get returned cars, those cars can be used for renting tomorrow
                returned_cars_first_loc = RETURNS_FIRST_LOC
                returned_cars_second_loc = RETURNS_SECOND_LOC
                num_of_cars_first_loc = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                num_of_cars_second_loc = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                returns += prob * (reward + DISCOUNT * state_value[num_of_cars_first_loc, num_of_cars_second_loc])
            else:
                for returned_cars_first_loc in range(0, POISSON_UPPER_BOUND):
                    for returned_cars_second_loc in range(0, POISSON_UPPER_BOUND):
                        num_of_cars_first_loc_ = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                        num_of_cars_second_loc_ = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                        prob_ = poisson(returned_cars_first_loc, RETURNS_FIRST_LOC) * \
                               poisson(returned_cars_second_loc, RETURNS_SECOND_LOC) * prob
                        returns += prob_ * (reward + DISCOUNT * state_value[num_of_cars_first_loc_, num_of_cars_second_loc_])
    return returns


In [2]:
MAX_CARS = 20

MAX_MOVE_OF_CARS = 5

GAMMA = 0.9

SHOP1 = 0
SHOP2 = 1
RENTAL = 0
RETURN = 1

VALUE_THRESHOLD = 1e-4

POISSON_UPPER_BOUND = 11

values = np.zeros((MAX_CARS + 1, MAX_CARS + 1))

poisson_cache = dict()
def poisson(n, lam):
    global poisson_cache
    key = n * 10 + lam
    if key not in poisson_cache.keys():
        poisson_cache[key] = exp(-lam) * pow(lam, n) / factorial(n)
    return poisson_cache[key]


poissonParameters = [
    [3, 3], # [rental, return]
    [4, 2]  # [rental, return]
]

def probabilityOfTransition(rentals_shop1, rentals_shop2, returns_shop1, returns_shop2):
    return poisson (poissonParameters[SHOP1][RENTAL], rentals_shop1) *\
           poisson (poissonParameters[SHOP2][RENTAL], rentals_shop2) *\
           poisson (poissonParameters[SHOP1][RETURN], returns_shop1) *\
           poisson (poissonParameters[SHOP2][RETURN], returns_shop2)


def stateValue(s, a, values):
    v = 0
    # this adds the new car moving restriction
    v -= max(0, abs(a)-1)*2
    for rentals_1 in range(POISSON_UPPER_BOUND):
        for rentals_2 in range(POISSON_UPPER_BOUND):
            cars_shop1 = s[SHOP1] - a
            cars_shop2 = s[SHOP2] + a

            rentals_1 = min(rentals_1, cars_shop1)
            rentals_2 = min(rentals_2, cars_shop1)

            earnings = (rentals_1 + rentals_2) * 10
            cars_shop1 -= rentals_1
            cars_shop2 -= rentals_2
            earnings -= max(cars_shop1 -  10, 0) * 4
            earnings -= max(cars_shop2 -  10, 0) * 4

            for returns_1 in range(POISSON_UPPER_BOUND):
                for returns_2 in range(POISSON_UPPER_BOUND):
                    cars_shop1 = min(returns_1 + cars_shop1, MAX_CARS)
                    cars_shop2 = min(returns_2 + cars_shop2, MAX_CARS)
                    p = probabilityOfTransition(rentals_1, rentals_2, returns_1, returns_2)
                    v += p * (earnings + values[cars_shop1, cars_shop2])

    return v

def updateValues(values, policy):
    for i in range(MAX_CARS+1):
        for j in range(MAX_CARS+1):
            a = policy[i, j]
            s1 = [i, j]
            values[i, j] = stateValue([i, j], policy[i, j], values)

    return values

value_change = 1
policy = np.zeros(values.shape, dtype=np.int)
policy = np.zeros(values.shape, dtype=np.int)

while value_change > VALUE_THRESHOLD:
    new_values = updateValues(np.copy(values), policy)
    value_change = np.abs((new_values - values)).sum()
    # print('value %f' % (np.abs(values).sum()))
    # print('new value %f' % (np.abs(new_values).sum()))
    print('value change %f' % (value_change))
    values = new_values

0.0
0.21627224821810828
0.2162732715978034
0.21627327160112028
0.21627327160112028
0.21627327160112028
0.21627327160112028
0.21627327160112028


KeyboardInterrupt: 